## Generating Word-level Features 

In [1]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

In [2]:
"""
Generates word embeddings for training and test and writes the embedding matrix to file.
"""
def generate_word_embeddings(i):
    print("Inside generate_word_embeddings")
    # Extracted the text for word embeddings    
    df_X_train= pd.read_pickle('data-files\\X_train_' + str(i) + '.pkl')
    df_X_test= pd.read_pickle('data-files\\X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('data-files\\Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('data-files\\Y_test_' + str(i) + '.pkl')
    df_train = pd.concat([df_X_train,df_Y_train], axis=1)
    df_test = pd.concat([df_X_test,df_Y_test], axis=1)
    
    train = df_train['Content'].fillna("_na_").values
    test = df_test['Content'].fillna("_na_").values
    
    """
    Fitting the tokenizer on training data and then using it to generate X_train and X_test
    NEEDS TO BE RERUN if training and test are changed
    """
    tokenizer = Tokenizer(num_words=input_length)
    tokenizer.fit_on_texts(list(train))
    train_tokenized = tokenizer.texts_to_sequences(train)
    test_tokenized = tokenizer.texts_to_sequences(test)
    X_train_we = pad_sequences(train_tokenized, maxlen=input_length, padding='post')
    X_test_we = pad_sequences(test_tokenized, maxlen=input_length, padding='post')
    
    print("Printing embedded X_train:", X_train_we.shape, X_train_we)
    
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))
    
    all_embeddings = np.stack(embeddings_index.values())
    embeddings_mean,embeddings_std = all_embeddings.mean(), all_embeddings.std()
    print(embeddings_mean,embeddings_std)
    
    word_index = tokenizer.word_index
    print(len(word_index), type(word_index))
    
    nb_words = min(vocab_size, len(word_index))
    print(nb_words)
    
    embedding_matrix = np.random.normal(embeddings_mean, embeddings_std, (nb_words, embed_size))
    #print(len(embedding_matrix), type(embedding_matrix), embedding_matrix.shape, embedding_matrix[1].dtype)
    
    for word, index in word_index.items(): # for all words in the dictioanry
        if index >= nb_words: continue # if the word index is greater than the vocabulary size, ignore and fetch next word
        embedding_vector = embeddings_index.get(word) # get the embedding vector for the word
        if embedding_vector is not None: embedding_matrix[index] = embedding_vector # add the vector to the embedding matrix  
    

    #Writing the word embedding matrix for future use
    f = open("output\\word-bilstm\\word_embedding_matrix_"+ str(i) + ".txt", "bw")
    embedding_matrix.tofile(f)
    
    return train, test, X_train_we, X_test_we, nb_words, embedding_matrix, input_length

## Generating NLP Features 

In [3]:
def encode_nlp_features(df, name, i):
    print("Inside encode_nlp_features")
    mlb = MultiLabelBinarizer()
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("ne")), columns=mlb.classes_, index=df.index).add_prefix('ne_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("iob")), columns=mlb.classes_, index=df.index).add_prefix('iob_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("alpha")), columns=mlb.classes_, index=df.index).add_prefix('al_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("digit")), columns=mlb.classes_, index=df.index).add_prefix('dig_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("punc")), columns=mlb.classes_, index=df.index).add_prefix('punc_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("url")), columns=mlb.classes_, index=df.index).add_prefix('url_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("oov")), columns=mlb.classes_, index=df.index).add_prefix('oov_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("stop")), columns=mlb.classes_, index=df.index).add_prefix('stop_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("coarsepos")), columns=mlb.classes_, index=df.index).add_prefix('cpos_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("finepos")), columns=mlb.classes_, index=df.index).add_prefix('fpos_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("dep")), columns=mlb.classes_, index=df.index).add_prefix('dep_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("offset")), columns=mlb.classes_, index=df.index).add_prefix('off_'))], axis=1)
    #print(df.shape)
    #print(df.columns.values.tolist())
    df.to_pickle('output\\nlp-meta-attn\\df_' + name + '_nlp_features_' + str(i) + '.pkl')
    return df

In [4]:
"""
This method should generate all the natural language processing features
"""
def generate_nlp_features(df, i, name=""):
    print("Inside generate_nlp_features")
    sentence_ne = []
    sentence_iob = []
    sentence_alpha = []
    sentence_isdigit = []
    sentence_ispunc = []
    sentence_isurl = []
    sentence_isoov = []
    sentence_isstop = []
    sentence_coarsepos = []
    sentence_finepos = []
    sentence_dep = []
    sentence_offset = []

    i=0
   
    """NEED TO CHANGE THE FUNCTIONAL COLUMN"""
    for entries in df["Content"]: #each entry is a sentence
        if i%100==0:
            print(i)
        i=i+1

        doc = nlp(entries) #Spacy function to get tags
        #print(doc)

        ne = []
        iob = []
        alpha = []
        isdigit = []
        ispunc = []
        isurl = []
        isoov = []
        isstop = []
        coarsepos = []
        finepos = []
        dep = []
        offset = []

        """
        For words in the sentence
        """
        for token in doc:
            """
            Named Entity Type
            """
            #print("NE Type:", token.ent_type)
            ne.append(token.ent_type_)

            """
            IOB code of named entity tag. B means the token begins an entity, 
            I means it is inside an entity, O means it is outside an entity, and 
            '' means no entity tag is set.
            """
            #print("IOB:", token.ent_iob) 
            iob.append(token.ent_iob_)

            """
            Does the token contain alphabetic characters?
            """
            #print("Alpha:", token.is_alpha)
            alpha.append(str(token.is_alpha))

            """
            Does the token contain digits?
            """
            #print("Digits:", token.is_digit)
            isdigit.append(str(token.is_digit))

            """
            Is the token punctuation?
            """
            #print("Punc:", token.is_punct)
            ispunc.append(str(token.is_punct))

            """
            Is the token like urls?
            """
            #print("Url:", token.like_url)
            isurl.append(str(token.like_url))

            """
            Is the token out-of-vocabulary?
            """
            #print("OOV:", token.is_oov)
            isoov.append(str(token.is_oov))

            """
            Is the token a stopword?
            """
            #print("Stop:", token.is_stop)
            isstop.append(str(token.is_stop))

            """
            Coarse-grained POS
            """
            #print("Coarse POS:", token.pos_)
            coarsepos.append(token.pos_)

            """
            Fine-grained POS
            """
            #print("Fine POS:", token.tag_)
            finepos.append(token.tag_)

            """
            Syntactic Dependency Relation
            """
            #print("Dependency:", token.dep_)
            dep.append(token.dep_)

            """
            Character offset
            """
            #print("Offset:", token.idx)
            coff = token.idx
            if(coff<100):
                off = 1
            elif(coff<200): 
                off =2
            elif(coff<300): 
                off =3
            elif(coff<400): 
                off =4
            elif(coff<500): 
                off =5
            elif(coff<600): 
                off =6
            elif(coff<700): 
                off =7
            elif(coff<800): 
                off =8
            elif(coff<900): 
                off =9
            elif(coff<1000): 
                off =10
            else: 
                off =11
            offset.append(off)
            
            #print(type(token.is_alpha))
            #break

        sentence_ne.append(ne)
        sentence_iob.append(iob)
        sentence_alpha.append(alpha)
        sentence_isdigit.append(isdigit)
        sentence_ispunc.append(ispunc)
        sentence_isurl.append(isurl)
        sentence_isoov.append(isoov)
        sentence_isstop.append(isstop)
        sentence_coarsepos.append(coarsepos)
        sentence_finepos.append(finepos)
        sentence_dep.append(dep)
        sentence_offset.append(offset)
    df["ne"] = sentence_ne
    df["iob"] = sentence_iob
    df["alpha"] = sentence_alpha
    df["digit"] = sentence_isdigit
    df["punc"] = sentence_ispunc
    df["url"] = sentence_isurl
    df["oov"] = sentence_isoov
    df["stop"] = sentence_isstop
    df["coarsepos"] = sentence_coarsepos
    df["finepos"] = sentence_finepos
    df["dep"] = sentence_dep
    df["offset"] = sentence_offset
    
    print(len(sentence_ne) == len(sentence_iob) == len(sentence_alpha) == len(sentence_isdigit) == \
          len(sentence_ispunc) == len(sentence_isurl) == len(sentence_isoov) == len(sentence_isstop) \
          == len(sentence_coarsepos) == len(sentence_finepos) == len(sentence_dep) == len(sentence_offset))

    df.to_pickle('output\\nlp-meta-attn\\df_' + name + '_nlp_features_intermediate_' + str(i) + '.pkl')
    return df

In [5]:
def get_nlp_encodings(i):
    """
    Obtain the dataset
    """
    print("Inside get_nlp_encodings")
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('data-files\\X_train_' + str(i) + '.pkl')
    df_X_test= pd.read_pickle('data-files\\X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('data-files\\Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('data-files\\Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    """
    merging the output labels to predictors
    """
    df_train = pd.concat([df_X_train,df_Y_train], axis=1)
    df_test = pd.concat([df_X_test,df_Y_test], axis=1)
    print("The size of training and test sets:", df_train.shape, df_test.shape)
    r,c = df_train.shape
    #print("Rows in training data =", r)
    """
    merging training and test data to one dataframe
    """
    df = pd.concat([df_train,df_test])
    #print("Final df:", df.shape)
    #print(df.head(2))
    """
    generating the nlp features using spacy
    """
    df_nlp = generate_nlp_features(df, i, 'all')
    #print("After-spacy:", df_nlp.shape, df_nlp.columns.values)
    #print(df_nlp.head(2))
    
    """
    encoding the nlp features using multilabelbinarizer
    """
    df_nlp_enc = encode_nlp_features(df_nlp, 'all_nlp_enc', i)
    #print(df_nlp_enc.shape)
    #print(df_nlp_enc.head(2))
    #print(df_nlp_enc.shape, df_nlp_enc.columns.values.tolist())
    df_nlp_enc.drop(['Content', 'Utterance_Number', 'Duration', 'Speaker', 'System_Number', 'Search_Task', 'Previous_User_Utterance', 'Previous_User_Speech_Act', 'Previous_Speech_Act', 'Previous_Search_Act', 'Speech_acts'], axis=1, inplace=True)
                     #['Transcript', 'Query_counter', 'Length', 'If_Intermediary', 'Complexity', 'Speech_acts'], axis=1, inplace=True)
    #print(df_nlp_enc.shape, df_nlp_enc.columns.values.tolist())
    train_split = df_nlp_enc.iloc[:r,:]
    test_split = df_nlp_enc.iloc[r:,:]
    print("Size after split:",train_split.shape, test_split.shape, df_Y_train.shape, df_Y_test.shape)
    return train_split, test_split, df_Y_train, df_Y_test

## Generating Metadata Features 

In [6]:
def get_metadata_features(i):
    print("Inside get_metadata_features")
    """
    Obtain the dataset
    """
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('data-files\\X_train_' + str(i) + '.pkl')
    #print(df_X_train.head())
    df_X_test= pd.read_pickle('data-files\\X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('data-files\\Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('data-files\\Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    X_train_meta = df_X_train.drop(['Content', 'Previous_User_Utterance'], axis=1)
    X_test_meta = df_X_test.drop(['Content', 'Previous_User_Utterance'], axis=1)
    
    #print(X_train_meta['Previous_Speech_Act'].unique())
    #print(X_train_meta['Previous_Search_Act'].unique())
    
    return X_train_meta, X_test_meta, df_Y_train, df_Y_test

In [7]:
from keras.engine.topology import Layer

class AttentionLayer(Layer):
   
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

Using TensorFlow backend.


In [8]:
def create_bilstm_nlp_meta_channels(reshaped_data_nlp, reshaped_data_meta):
    samples,timesteps,features = reshaped_data_nlp.shape
    inp_nlp = Input(shape=(timesteps,features), name='input_nlp')
    # lstm needs (samples,timesteps,features) tensor as the input
    x1 = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(inp_nlp) 
    x1, sent_coeffs1 = AttentionLayer(features,return_coefficients=True,name='sent_attention1')(x1)
    #x1 = GlobalMaxPool1D()(x1)
    #x1 = Dense(100, activation="relu")(x1)
    x1 = Dropout(0.25)(x1)
    x1 = Dense(12, activation="softmax")(x1) #output layer  
    
    samples,timesteps,features = reshaped_data_meta.shape
    inp_meta = Input(shape=(timesteps,features), name='input_meta')
    # lstm needs (samples,timesteps,features) tensor as the input
    x3 = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(inp_meta) 
    x3, sent_coeffs3 = AttentionLayer(features,return_coefficients=True,name='sent_attention3')(x3)
    #x3 = GlobalMaxPool1D()(x3)
    #x3 = Dense(100, activation="relu")(x3)
    x3 = Dropout(0.25)(x3)
    x3 = Dense(12, activation="softmax")(x3) #output layer
    
    
    """
    Merging outputs of nlp and meta
    """
    merge_nlp_meta = concatenate([x1, x3])
    output_nlp_meta = Dense(12, activation="softmax")(merge_nlp_meta)
    model_nlp_meta_combined = Model(inputs=[inp_nlp, inp_meta], outputs=output_nlp_meta)
    
    return model_nlp_meta_combined

In [9]:
def execute_bilstm_nlp_meta_channel(i):
    """
    nlp data
    """
    df_train_nlp_encodings, df_test_nlp_encodings, df_Y_train_nlp, df_Y_test_nlp = get_nlp_encodings(i)
    train_nlp_enc = df_train_nlp_encodings.values
    test_nlp_enc = df_test_nlp_encodings.values
    
    
    timestamp = 1 #number of successive sequences combined
    
    r, c = df_train_nlp_encodings.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped_nlp = np.concatenate([train_nlp_enc[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped_nlp = X_train_reshaped_nlp.reshape(staggering, timestamp, c) # c is the number of features
    
    print("NLP data\n------------------")
    print("Training set\n------------------")
    print(X_train_reshaped_nlp)
    print(X_train_reshaped_nlp.shape)
    
    r2, c2 = df_test_nlp_encodings.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped_nlp = np.concatenate([test_nlp_enc[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped_nlp = X_test_reshaped_nlp.reshape(staggering2, timestamp, c2) # c is the number of features
    
    print("NLP data\n------------------")
    print("Test set\n------------------")
    print(X_test_reshaped_nlp)
    print(X_test_reshaped_nlp.shape)
    
    df_Y_train_nlp = df_Y_train_nlp.iloc[timestamp-1:,]
    df_Y_test_nlp = df_Y_test_nlp.iloc[timestamp-1:,]
    
    print("Output Labels\n------------------")
    print(df_Y_train_nlp.shape, df_Y_test_nlp.shape)    
    
    
    """
    meta data
    """
    df_train_meta, df_test_meta, df_Y_train_meta, df_Y_test_meta = get_metadata_features(i)
    train_meta = df_train_meta.values
    test_meta = df_test_meta.values
    
    timestamp = 1 #number of successive sequences combined
    
    r, c = df_train_meta.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped_meta = np.concatenate([train_meta[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped_meta = X_train_reshaped_meta.reshape(staggering, timestamp, c) # c is the number of features
   
    r2, c2 = df_test_meta.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped_meta = np.concatenate([test_meta[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped_meta = X_test_reshaped_meta.reshape(staggering2, timestamp, c2) # c is the number of features
    
    print("Meta data\n------------------")
    print("Training set\n------------------")
    print(X_train_reshaped_meta)
    print(X_train_reshaped_meta.shape)
    
    print("Meta data\n------------------")
    print("Test set\n------------------")
    print(X_test_reshaped_meta)
    print(X_test_reshaped_meta.shape)
    
    df_Y_train_meta = df_Y_train_meta.iloc[timestamp-1:,]
    df_Y_test_meta = df_Y_test_meta.iloc[timestamp-1:,]
    
    print("Output Labels\n------------------")
    print(df_Y_train_meta.shape, df_Y_test_meta.shape)
    
    # ----------------------------------------------------
    # can save and read back for iterations
    # ----------------------------------------------------
    """
    $$$$ ---------------
    $$$$ CHANGE HERE _word_meta _word_nlp _nlp_meta _word_nlp_meta
    $$$$ ---------------
    """
    
    model_nlp_meta = create_bilstm_nlp_meta_channels(X_train_reshaped_nlp, X_train_reshaped_meta)
    model_nlp_meta.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    model_nlp_meta.summary()  
    
    #$$ this section is unique depending on the type of channel --starts
    """
    encoding the output labels
    """
    encoder = LabelEncoder()
    encoder.fit(df_Y_train_nlp)
    training_op_labels_encoded = encoder.transform(df_Y_train_nlp)
    test_op_labels_encoded = encoder.transform(df_Y_test_nlp)
    print("Output Labels\n-------------------")
    print(training_op_labels_encoded.shape, test_op_labels_encoded.shape)
    
    """
    converting the output labels to one-hot form
    """
    training_op_labels_onehot= np_utils.to_categorical(training_op_labels_encoded)
    test_op_labels_onehot = np_utils.to_categorical(test_op_labels_encoded)

    print(training_op_labels_onehot.shape, len(training_op_labels_onehot))
    print(test_op_labels_onehot.shape, len(test_op_labels_onehot))
    
    #$$ this section is unique depending on the type of channel -ends
    
    
    
    # ----------------------------------------------------
    # can save and read back for iterations
    # X_train_reshaped_nlp, X_test_reshaped_nlp
    # X_train_we, X_test_we
    # training_op_labels_onehot, test_op_labels_onehot
    # ----------------------------------------------------
    """
    $$$$ ---------------
    $$$$ CHANGE HERE _word_meta _word_nlp _nlp_meta _word_nlp_meta
    $$$$ ---------------
    """
    #change the input_
    #train data
    model_nlp_meta_train = model_nlp_meta.fit({'input_nlp': X_train_reshaped_nlp, 'input_meta':X_train_reshaped_meta}, \
                                  training_op_labels_onehot, \
                                  batch_size=BATCH_SIZE, \
                                  epochs=EPOCHS, \
                                  validation_split=0.1)
    # load model if required 
    # compile model
    model_nlp_meta.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    
    # evaluate model
    print(model_nlp_meta.metrics_names)
    #change the input_
    loss, binary_accuracy, categorical_accuracy = model_nlp_meta.evaluate({'input_nlp': X_test_reshaped_nlp, 'input_meta':X_test_reshaped_meta},\
                                                                    y = test_op_labels_onehot,\
                                                                    batch_size=BATCH_SIZE,\
                                                                    verbose=1)
    print(loss, binary_accuracy, categorical_accuracy)
    

    """
    predict the probabilty of output classes
    and pick the best one
    """
    #change the input_
    prediction_vector = model_nlp_meta.predict({'input_nlp': X_test_reshaped_nlp, 'input_meta':X_test_reshaped_meta}, \
                           batch_size=BATCH_SIZE,\
                           verbose=1)
    predicted_classes = np.argmax(prediction_vector, axis=1)
    original_classes = np.argmax(test_op_labels_onehot, axis=1)
    
    """
    # verification of correctness:
    total_correct = sum(original_classes == predicted_classes)
    print("Total number of correct predictions:",total_correct)
    print("Accuracy:",total_correct/len(test_op_labels_onehot))
    acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
    print('Overall accuracy: {} %'.format(acc*100))
    """
    conf_mat = confusion_matrix(predicted_classes, original_classes)
    print("\n", conf_mat, "\n")
    
    return predicted_classes, categorical_accuracy, binary_accuracy, conf_mat

In [10]:
"""
import tensorflow as tf
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True
"""

import numpy as np 
import pandas as pd
from numpy import array

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model, model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils, to_categorical
from keras.engine.topology import Layer
from keras import backend as K

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

# checkpoint
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Attention

"""
Required for NLP model
"""
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()   

# set parameters for word embeddings
embed_size = 100 # how big is each word vector
vocab_size = 25000 # how many unique words to use (i.e num rows in embedding vector) max
input_length = 100 # max number of words in the input 

#set parameters for bilstm
BATCH_SIZE=32
EPOCHS=300

EMBEDDING_FILE='glove.6B.100d.txt'    

file = open('output\\nlp-meta-attn\\bilstm_nlp_meta_attn_op.txt','w') #overwrites previous
file.close()

if __name__== "__main__":
    df_prediction = pd.DataFrame()
    df_accuracy =  pd.DataFrame()
    file = open('output\\nlp-meta-attn\\bilstm_nlp_meta_attn_op.txt','a') #append mode 
    """
    Change the range before executing
    """
    for i in range(1,31):
        outputname = 'nlp_meta_attn'+ str(i)        
        predictions, acc_cat, acc_bin, conf_matrix = execute_bilstm_nlp_meta_channel(i)
        df_prediction[outputname] = predictions
        df_accuracy[i] = [acc_cat]
        file.write("\nIteration:" + str(i) + "\nCategorical Accuracy:" + str(acc_cat) + 
                   "\nBinary Accuracy:" + str(acc_bin) + "\nConfusion Matrix:\n" + str(conf_matrix) + "\n\n")
        df_prediction.to_csv('output\\nlp-meta-attn\\predictions_bilstm_nlp_meta_attn_' + str(i) + '.csv')    
        df_accuracy.to_csv('output\\nlp-meta-attn\\accuracy_bilstm_nlp_meta_attn_' + str(i) + '.csv')    
    
    df_prediction.to_csv('output\\nlp-meta-attn\\predictions_bilstm_nlp_meta_attn.csv')    
    df_accuracy.to_csv('output\\nlp-meta-attn\\accuracy_bilstm_nlp_meta_attn.csv')    
    file.close()

Inside get_nlp_encodings
The size of training and test sets: (1467, 11) (367, 11)
Inside generate_nlp_features
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
True
Inside encode_nlp_features
Size after split: (1467, 148) (367, 148) (1467, 1) (367, 1)
NLP data
------------------
Training set
------------------
[[[1 0 0 ... 0 0 0]]

 [[1 1 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]]

 ...

 [[1 0 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]]]
(1467, 1, 148)
NLP data
------------------
Test set
------------------
[[[1 0 0 ... 0 0 0]]

 [[1 1 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]]

 ...

 [[1 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]]]
(367, 1, 148)
Output Labels
------------------
(1467, 1) (367, 1)
Inside get_metadata_features
Meta data
------------------
Training set
------------------
[[[67.0 11.0 0 ... 1 1 3]]

 [[19.0 13.0 0 ... 6 4 3]]

 [[13.0 67.0 0 ... 1 1 3]]

 ...

 [[65.0 15.0 0 ... 10 4 3]]

 [[29.0 5.0 0 ... 1 1 2]]

 [[0.

C:\Users\iSchool User\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)



Train on 1320 samples, validate on 147 samples
Epoch 1/300
1320/1320 [==============================] - 1s 783us/step - loss: 2.4544 - binary_accuracy: 0.9167 - categorical_accuracy: 0.1174 - val_loss: 2.3758 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.1769
Epoch 2/300
1320/1320 [==============================] - 0s 117us/step - loss: 2.3021 - binary_accuracy: 0.9167 - categorical_accuracy: 0.1879 - val_loss: 2.2156 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.1769
Epoch 3/300
1320/1320 [==============================] - 0s 84us/step - loss: 2.1776 - binary_accuracy: 0.9167 - categorical_accuracy: 0.2015 - val_loss: 2.0845 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.2177
Epoch 4/300
1320/1320 [==============================] - 0s 95us/step - loss: 2.0794 - binary_accuracy: 0.9167 - categorical_accuracy: 0.2727 - val_loss: 1.9754 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.3673
Epoch 5/300
1320/1320 [=======================

Epoch 36/300
1320/1320 [==============================] - 0s 93us/step - loss: 0.9658 - binary_accuracy: 0.9533 - categorical_accuracy: 0.7841 - val_loss: 0.8390 - val_binary_accuracy: 0.9649 - val_categorical_accuracy: 0.8299
Epoch 37/300
1320/1320 [==============================] - 0s 90us/step - loss: 0.9520 - binary_accuracy: 0.9549 - categorical_accuracy: 0.7841 - val_loss: 0.8293 - val_binary_accuracy: 0.9637 - val_categorical_accuracy: 0.8231
Epoch 38/300
1320/1320 [==============================] - 0s 87us/step - loss: 0.9469 - binary_accuracy: 0.9544 - categorical_accuracy: 0.7727 - val_loss: 0.8258 - val_binary_accuracy: 0.9666 - val_categorical_accuracy: 0.8095
Epoch 39/300
1320/1320 [==============================] - 0s 86us/step - loss: 0.9315 - binary_accuracy: 0.9579 - categorical_accuracy: 0.7833 - val_loss: 0.8169 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.8231
Epoch 40/300
1320/1320 [==============================] - 0s 83us/step - loss: 0.9048 - bina

1320/1320 [==============================] - 0s 87us/step - loss: 0.5837 - binary_accuracy: 0.9720 - categorical_accuracy: 0.8250 - val_loss: 0.5961 - val_binary_accuracy: 0.9717 - val_categorical_accuracy: 0.8299
Epoch 109/300
1320/1320 [==============================] - 0s 97us/step - loss: 0.6157 - binary_accuracy: 0.9699 - categorical_accuracy: 0.8212 - val_loss: 0.6225 - val_binary_accuracy: 0.9705 - val_categorical_accuracy: 0.8163
Epoch 110/300
1320/1320 [==============================] - 0s 92us/step - loss: 0.5931 - binary_accuracy: 0.9712 - categorical_accuracy: 0.8205 - val_loss: 0.6395 - val_binary_accuracy: 0.9688 - val_categorical_accuracy: 0.8095
Epoch 111/300
1320/1320 [==============================] - 0s 91us/step - loss: 0.5866 - binary_accuracy: 0.9712 - categorical_accuracy: 0.8227 - val_loss: 0.6095 - val_binary_accuracy: 0.9700 - val_categorical_accuracy: 0.8299
Epoch 112/300
1320/1320 [==============================] - 0s 94us/step - loss: 0.5800 - binary_accura

...

Epoch 72/300
1320/1320 [==============================] - 1s 475us/step - loss: 0.8584 - binary_accuracy: 0.9538 - categorical_accuracy: 0.7258 - val_loss: 1.0226 - val_binary_accuracy: 0.9444 - val_categorical_accuracy: 0.6327
Epoch 73/300
1320/1320 [==============================] - 1s 457us/step - loss: 0.8719 - binary_accuracy: 0.9533 - categorical_accuracy: 0.7167 - val_loss: 1.0398 - val_binary_accuracy: 0.9439 - val_categorical_accuracy: 0.6259
Epoch 74/300
1320/1320 [==============================] - 1s 460us/step - loss: 0.8494 - binary_accuracy: 0.9541 - categorical_accuracy: 0.7205 - val_loss: 1.0245 - val_binary_accuracy: 0.9461 - val_categorical_accuracy: 0.6259
Epoch 75/300
1320/1320 [==============================] - 1s 465us/step - loss: 0.8580 - binary_accuracy: 0.9539 - categorical_accuracy: 0.7167 - val_loss: 1.0143 - val_binary_accuracy: 0.9422 - val_categorical_accuracy: 0.6327
Epoch 76/300
1320/1320 [==============================] - 1s 551us/step - loss: 0.8665 -

Epoch 108/300
1320/1320 [==============================] - 1s 510us/step - loss: 0.7061 - binary_accuracy: 0.9660 - categorical_accuracy: 0.7970 - val_loss: 0.8845 - val_binary_accuracy: 0.9512 - val_categorical_accuracy: 0.7279
Epoch 109/300
1320/1320 [==============================] - 1s 517us/step - loss: 0.6929 - binary_accuracy: 0.9651 - categorical_accuracy: 0.8068 - val_loss: 0.8821 - val_binary_accuracy: 0.9495 - val_categorical_accuracy: 0.7211
Epoch 110/300
1320/1320 [==============================] - 1s 778us/step - loss: 0.7085 - binary_accuracy: 0.9643 - categorical_accuracy: 0.7886 - val_loss: 0.8824 - val_binary_accuracy: 0.9501 - val_categorical_accuracy: 0.7007
Epoch 111/300
1320/1320 [==============================] - 1s 479us/step - loss: 0.7031 - binary_accuracy: 0.9653 - categorical_accuracy: 0.7947 - val_loss: 0.8785 - val_binary_accuracy: 0.9518 - val_categorical_accuracy: 0.7279
Epoch 112/300
1320/1320 [==============================] - 1s 481us/step - loss: 0.6

Epoch 179/300
1320/1320 [==============================] - 1s 477us/step - loss: 0.5103 - binary_accuracy: 0.9794 - categorical_accuracy: 0.8659 - val_loss: 0.7149 - val_binary_accuracy: 0.9683 - val_categorical_accuracy: 0.7687
Epoch 180/300
1320/1320 [==============================] - 1s 455us/step - loss: 0.5083 - binary_accuracy: 0.9777 - categorical_accuracy: 0.8644 - val_loss: 0.7081 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.7823
Epoch 181/300
1320/1320 [==============================] - 1s 488us/step - loss: 0.5176 - binary_accuracy: 0.9783 - categorical_accuracy: 0.8697 - val_loss: 0.7097 - val_binary_accuracy: 0.9683 - val_categorical_accuracy: 0.7619
Epoch 182/300
1320/1320 [==============================] - 1s 481us/step - loss: 0.5204 - binary_accuracy: 0.9772 - categorical_accuracy: 0.8591 - val_loss: 0.7110 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.7619
Epoch 183/300
1320/1320 [==============================] - 1s 513us/step - loss: 0.4

Epoch 215/300
1320/1320 [==============================] - 1s 465us/step - loss: 0.4571 - binary_accuracy: 0.9809 - categorical_accuracy: 0.8848 - val_loss: 0.6660 - val_binary_accuracy: 0.9711 - val_categorical_accuracy: 0.8163
Epoch 216/300
1320/1320 [==============================] - 1s 465us/step - loss: 0.4478 - binary_accuracy: 0.9817 - categorical_accuracy: 0.8902 - val_loss: 0.6702 - val_binary_accuracy: 0.9722 - val_categorical_accuracy: 0.8163
Epoch 217/300
1320/1320 [==============================] - 1s 461us/step - loss: 0.4354 - binary_accuracy: 0.9817 - categorical_accuracy: 0.8879 - val_loss: 0.6741 - val_binary_accuracy: 0.9717 - val_categorical_accuracy: 0.8163
Epoch 218/300
1320/1320 [==============================] - 1s 455us/step - loss: 0.4259 - binary_accuracy: 0.9826 - categorical_accuracy: 0.8947 - val_loss: 0.6875 - val_binary_accuracy: 0.9705 - val_categorical_accuracy: 0.8095
Epoch 219/300
1320/1320 [==============================] - 1s 469us/step - loss: 0.4

Epoch 286/300
1320/1320 [==============================] - 1s 481us/step - loss: 0.3908 - binary_accuracy: 0.9826 - categorical_accuracy: 0.8977 - val_loss: 0.6743 - val_binary_accuracy: 0.9705 - val_categorical_accuracy: 0.8095
Epoch 287/300
1320/1320 [==============================] - 1s 461us/step - loss: 0.3644 - binary_accuracy: 0.9835 - categorical_accuracy: 0.9000 - val_loss: 0.6734 - val_binary_accuracy: 0.9717 - val_categorical_accuracy: 0.8163
Epoch 288/300
1320/1320 [==============================] - 1s 462us/step - loss: 0.3569 - binary_accuracy: 0.9850 - categorical_accuracy: 0.9106 - val_loss: 0.6808 - val_binary_accuracy: 0.9717 - val_categorical_accuracy: 0.8163
Epoch 289/300
1320/1320 [==============================] - 1s 478us/step - loss: 0.3601 - binary_accuracy: 0.9859 - categorical_accuracy: 0.9114 - val_loss: 0.6557 - val_binary_accuracy: 0.9722 - val_categorical_accuracy: 0.8163
Epoch 290/300
1320/1320 [==============================] - 1s 467us/step - loss: 0.3